In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import missingno as msno
from matplotlib.pyplot import figure
import seaborn as sns
from scipy.stats import norm, skew 
from scipy import stats
from datetime import datetime
import scipy.stats
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score,f1_score
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.model_selection import cross_val_score
import statsmodels.api as sm
import warnings
warnings.filterwarnings(action="ignore")
from tqdm import tqdm
from statsmodels.tsa.ar_model import AutoReg,AR
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller,kpss
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import pacf
import json
from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree
from pymc3.stats import mcse
import arviz as az
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder
import pymc3 as pm

In [27]:
def check_adf(column):# return p value
    adf_result=adfuller(column)
    if adf_result[1]<=0.05:
        return 'Stationary'
    else:
        return 'Non-Stationary'
    

def get_diff_order(column):
  if check_adf(column)=='Non-Stationary':
    column=column.diff().dropna()
    if check_adf(column)=='Non-Stationary':
        diff_order=2
        column=column.diff().dropna()
    else:
        diff_order=1
  else:
    diff_order=0
  return column,diff_order

def find_Feature_lag(df,target,maxlag,diff_order,p):
    df_sample=df.copy()
    best_aic = np.inf
    best_lag = None
    max_lag=maxlag
    for i in range(1,max_lag+1):
        df_sample[target]=df_sample[target].shift(i)
        df_c=df_sample.dropna()
        model=SARIMAX(df_c[['unweighted_ili']],df_c[target],order=(p,diff_order,0))
        results=model.fit()
        if results.aic<best_aic and results.aic>0:
            best_aic=results.aic 
            best_lag=i
        if best_aic<0 or not best_lag:
            best_lag=2
    return best_lag

def _create_lags(data,maxlag,target,lag=None):
    datax=data.copy()
    for i in datax.columns:
        if i!='state':
          if lag:
            best_lag_value=lag
          #else:
            #best_lag_value=find_Feature_lag(data,i,maxlag,diff_order,p)
          datax[i + "_lag" + str(best_lag_value)]=datax[i].shift(best_lag_value)
          if i!=target:
            datax=datax.drop(i,axis='columns')
    return datax
def best_ili_lag(df,maxlag,diff_order):
    best_aic = np.inf
    best_lag = None
    max_lag=maxlag
    for i in range(1,max_lag+1):
        model=ARIMA(df[['unweighted_ili']],order=(i,1,0))
        results=model.fit()
        if results.aic<best_aic and results.aic>0:
            best_aic=results.aic 
            best_lag=i
        if best_aic<0 or not best_lag:
            best_lag=2
        return best_lag
def _selected_create_lags(data,target,lag_dict):
    datax=data.copy()
    for i in datax.columns:
        if i!='state':
            datax[i + "_lag" + str(lag_dict[i])]=datax[i].shift(lag_dict[i])
        if i!=target:
            datax=datax.drop(i,axis='columns')
    return datax

In [28]:
df=pd.read_csv('/kaggle/input/newdata/new_data.csv')
df['date']=pd.to_datetime(df['date'])
df1=(df.set_index('date',drop=True))
states=df1['state'].unique()
df1['school_mode'] = df1['school_mode'].round().astype(int)
df1['elderly_mode'] = df1['elderly_mode'].round().astype(int)
ordinal_encoder = OrdinalEncoder()
ordinal_encoded_values = ordinal_encoder.fit_transform(df1[['school_mode', 'elderly_mode']])
df1[['school_mode', 'elderly_mode']] = ordinal_encoded_values
print(df1.shape)
df1.head(10)

(5919, 12)


,state,elderly_mode,school_mode,unweighted_ili,mask,rr_av7,gp_av7,transit_av7,work_av7,resid_av7,ILIp,pca_1
date,,,,,,,,,,,,
2020-03-01,AL,0.0,0.0,6.32549,0.0000,12.857143,7.285714,10.285714,2.285714,-1.000000,0.006724,38.286813
2020-03-01,AK,0.0,0.0,3.70284,0.0000,11.571429,8.142857,8.857143,3.571429,-0.857143,0.010216,37.121842
2020-03-01,AZ,0.0,0.0,2.31816,0.0615,8.428571,6.000000,6.428571,2.571429,-1.000000,0.002891,32.924371
2020-03-01,AR,0.0,0.0,4.90157,0.0000,17.142857,10.000000,5.857143,3.285714,-2.142857,0.021699,37.846680
2020-03-01,CA,0.0,0.0,3.99147,0.1583,5.142857,5.000000,0.857143,2.428571,-0.714286,0.008007,26.642818
2020-03-01,CO,0.0,0.0,5.82979,0.1030,6.714286,7.714286,2.428571,3.571429,-0.857143,0.010756,29.702890
2020-03-01,CT,0.0,0.0,6.19415,0.0000,6.428571,4.571429,4.714286,2.857143,-0.857143,0.010896,30.288501
2020-03-01,DE,0.0,0.0,2.46723,0.0000,9.714286,4.428571,4.285714,3.142857,-0.857143,0.003577,31.425038
2020-03-01,GA,0.0,0.0,5.44455,0.0001,8.857143,5.142857,4.428571,1.857143,-1.000000,0.007361,31.086173


In [29]:
def clean(dataframe):
  Q1 = dataframe['unweighted_ili'].quantile(0.25)
  Q3 = dataframe['unweighted_ili'].quantile(0.75)
  IQR = Q3 - Q1
  median = dataframe['unweighted_ili'].median()
  dataframe = dataframe[(dataframe['unweighted_ili'] >0) & (dataframe['unweighted_ili'] <= median + 1.5*IQR)]
  return dataframe
def return_state_data(data,state,start_date,end_date,cols,target_variable,lag_dict=None,own_lag_selection=True,lagg=None):
  df1_st=data[data['state']==state][start_date:end_date]
  df2_st=df1_st[cols]
  if own_lag_selection:
    data_new=_selected_create_lags(data=df2_st.dropna(),target=target_variable,lag_dict=lag_dict)
  else:
      data_new=_create_lags(data=df2_st.dropna(),maxlag=4,target=target_variable,lag=lagg)# p wnt work here, as i have put lagg even for ili-lagg
  df_c1=data_new.dropna()
  return df_c1
def compute_log_model_evidence(trace, model):
    logp = 0
    for i in range(trace.nchains):
        logp += np.sum(model.logp(trace.point(i)))
    logp /= trace.nchains
    logp_prior = model.logp(model.test_point).sum()

In [71]:
class Model:
    def __init__(self, county_idxs, new_dat_mat, coords,categorical=True):
        self.county_idxs = county_idxs
        self.new_dat_mat = new_dat_mat
        self.coords = coords
        self.model = None
        self.trace = None
        if categorical:
            self.categorical=categorical
        self.categorical_school=new_dat_mat[:,2]
        self.categorical_elderly=new_dat_mat[:,1]
        self.school_mode_values = np.unique(self.categorical_school)
        self.school_mode_idx = np.searchsorted(self.school_mode_values , self.categorical_school)
        self.elderly_mode_values = np.unique(self.categorical_elderly)
        self.elderly_mode_idx = np.searchsorted(self.elderly_mode_values, self.categorical_elderly)
    def build_model(self,model_distribution):
        with pm.Model(coords=self.coords) as hierarchical_model:
            #PRIORS
            #b_pca_= pm.Normal('b_pca_', mu=0., sigma=1)
            b_masking= pm.Normal('b_masking', mu=0., sigma=1)
            b_resid= pm.Normal('b_resid', mu=0., sigma=1)
            b_gp= pm.Normal('b_gp', mu=0., sigma=1)
            b_work= pm.Normal('b_work', mu=0., sigma=1)
            b_transit= pm.Normal('b_transit', mu=0., sigma=1)
            b_rr= pm.Normal('b_rr', mu=0., sigma=1)

            if  hasattr(self, 'categorical') and self.categorical:
                b_school= pm.Normal('b_school', mu=0., sigma=1,dims='school_mode' )
                school_category = pm.Data('school_mode', self.categorical_school, dims='obs_id')
                b_elderly= pm.Normal('b_elderly', mu=0., sigma=1,dims='elderly_mode')
                elderly_category = pm.Data('elderly_mode', self.categorical_elderly, dims='obs_id')
            else:
                b_school= pm.Normal('b_school', mu=0., sigma=1)
                b_elderly= pm.Normal('b_elderly', mu=0., sigma=1)
            mu_a = pm.Normal('mu_a', mu=0., sigma=1)
            sigma_a = pm.HalfCauchy('sigma_a', beta=1)

            #hierarchical part
            county_idx = pm.Data("county_idx",self.county_idxs , dims="obs_id")
            floor = pm.Data("floor", self.new_dat_mat[:,0], dims="obs_id")
            b_ili_lag = pm.Normal(f'ili_lag_{county_idx}', mu=0, sigma=1,dims="county")
            a = pm.Normal(f'alpha_{county_idx}', mu=mu_a, sigma=sigma_a,dims="county")
            if hasattr(self, 'categorical') and self.categorical:
                formula= (a[county_idx] + b_ili_lag[county_idx]*self.new_dat_mat[:,3] + b_masking*self.new_dat_mat[:,4]+
                b_school[self.school_mode_idx]*self.new_dat_mat[:,2] + b_elderly[self.elderly_mode_idx]*self.new_dat_mat[:,1]
                +b_resid*self.new_dat_mat[:,5] + b_gp*self.new_dat_mat[:,7] + b_work*self.new_dat_mat[:,9] + b_transit*self.new_dat_mat[:,8]+
                    b_rr*self.new_dat_mat[:,6] )
                
            else:
                formula= (a[county_idx] + b_ili_lag[county_idx]*self.new_dat_mat[:,3] + b_masking*self.new_dat_mat[:,4]+
                         b_school*self.new_dat_mat[:,2] + b_elderly*self.new_dat_mat[:,1]+b_pca_*self.new_dat_mat[:,5])
            eps = pm.HalfCauchy("eps", 1)
            if model_distribution=='Poisson':
                pm.Poisson(f'y_like_{county_idx}', mu=np.exp(formula), observed=floor)
            elif model_distribution=='Normal':
                pm.Normal(f'y_like_{county_idx}', mu=formula,sigma=eps,observed=floor) 
    
            self.model = hierarchical_model

    def fit_model(self,model_distribution, draws=5000, step=None):
        if self.model is None:
            self.build_model(model_distribution)

        with self.model:
            if step=='Metropolis':
                self.trace = pm.sample(draws=draws, step=pm.Metropolis())
            elif step=='Nuts':
                self.trace=pm.sample(draws=draws)
        return self.trace,self.model

    def plot_trace(self,cols):
        if self.trace is None:
            raise ValueError("Trace is not available. You need to fit the model first.")

        az.plot_posterior(self.trace, var_names=cols,kind='hist')
    def get_posteriors(self):
        return pm.sample_posterior_predictive(self.trace, model=self.model)
    def get_waic(self):
        waic=pm.waic(self.trace, self.model)
        return round(waic.waic,3)
    def get_summary(self):
        return pm.summary(self.trace)[['mean','sd','hdi_3%','hdi_97%']]
    def get_bic(self):
        log_likelihood=compute_log_model_evidence(self.trace,self.model)
        bic = -2 * log_likelihood + np.log(len(self.new_dat_mat)) * (model_graph.ndim)
        return bic

In [46]:
def transformed_data(dataframe,no_of_columns,start_date,end_date,cols,target_variable,lag_dict=None,lagg=None): 
    dat_mat = np.zeros((len(states)*100,no_of_columns+1))
    state_idx=1
    for state in states: 
        if state not in ['FL']: 
            if lag_dict and not lagg:
                tmp=return_state_data(dataframe,state,start_date,end_date,cols,target_variable,lag_dict=lag_dict,own_lag_selection=True).to_numpy()
            else:
                tmp = return_state_data(dataframe,state,lagg,start_date,end_date,cols,target_variable,lagg=lagg).to_numpy()
            tmp = np.concatenate((tmp,np.repeat(state_idx,tmp.shape[0]).reshape((-1,1))),axis=1)
            dat_mat = np.append(dat_mat,tmp,axis=0)

            state_idx += 1
    new_dat_mat = dat_mat[dat_mat[:, no_of_columns] != 0]
    county_idxs, counties = pd.factorize(new_dat_mat[:,no_of_columns])
    n_school_modes = len(np.unique(new_dat_mat[:, 2]))
    n_elderly_modes = len(np.unique(new_dat_mat[:, 1]))
    coords = {
        "county": counties,
        "obs_id": range(new_dat_mat.shape[0]),
        "school_mode":np.arange(0,4),
        "elderly_mode":np.arange(0,4)
    }
    return new_dat_mat,coords,county_idxs

In [51]:
s=return_state_data(df1,'TX','2020-01-01','2022-06-26',['elderly_mode', 'school_mode', 'unweighted_ili', 'mask', 'resid_av7',
                     'rr_av7','gp_av7','transit_av7','work_av7'],'unweighted_ili',lag_dict=baseline_ili_lag_dict,own_lag_selection=True).to_numpy()

In [74]:
baseline_ili_lag_dict={'mask':1,'unweighted_ili':1,'elderly_mode':1,'school_mode':1,'resid_av7':1,'rr_av7':1,'gp_av7':1,'transit_av7':1,'work_av7':1}
new_dat_mat, coords, county_idxs = transformed_data(dataframe=df1, no_of_columns=10,
                        start_date='2020-01-01', end_date='2022-06-26',
                cols=['elderly_mode', 'school_mode', 'unweighted_ili', 'mask', 'resid_av7',
                     'rr_av7','gp_av7','transit_av7','work_av7'],target_variable='unweighted_ili',lag_dict=baseline_ili_lag_dict)
pd.DataFrame(new_dat_mat).head()

,0,1,2,3,4,5,6,7,8,9,10
0,6.10561,0.0,0.0,6.32549,0.0000,-1.000000,12.857143,7.285714,10.285714,2.285714,1.0
1,6.64586,0.0,0.0,6.10561,0.0000,-0.714286,9.857143,7.142857,10.285714,2.571429,1.0
2,4.64715,0.0,0.0,6.64586,0.0034,2.857143,-2.142857,21.714286,4.428571,-7.142857,1.0
3,3.23893,3.0,2.0,4.64715,0.1170,11.857143,-30.714286,1.142857,-16.000000,-28.571429,1.0
4,2.30453,3.0,3.0,3.23893,0.2133,13.285714,-33.000000,-5.142857,-24.142857,-34.857143,1.0


In [ ]:
perc_lag_dict={'mask':2,'percent_positive':4,'elderly_mode':4,'school_mode':3,'pca_1':4}
ili_lag_dict={'mask':4,'unweighted_ili':4,'elderly_mode':4,'school_mode':3,'pca_1':1}
ILIp_dict={'mask':4,'ILIp':1,'elderly_mode':4,'school_mode':1,'pca_1':4}
s=return_state_data(df1,'AL','2020-03-01','2022-06-26',cols=['elderly_mode', 'school_mode', 'unweighted_ili', 'mask', 'pca_1'],
                  target_variable='unweighted_ili',lag_dict=ili_lag_dict,own_lag_selection=True)

In [ ]:
"""df_summary = pd.DataFrame()
for i in range(1,5):
    print(i)
    new_dat_mat, coords, county_idxs = transformed_data(dataframe=df1, no_of_columns=6,
                        start_date='2020-03-01', end_date='2022-06-26',
                cols=['elderly_mode', 'school_mode', 'percent_positive', 'mask', 'pca_1'],target_variable='percent_positive',lag_dict=perc_lag_dict)
    model = Model(county_idxs, new_dat_mat, coords,categorical=True)    
    trace,model_graph= model.fit_model(model_distribution='Poisson', draws=10000, step='Metropolis')
    waic = model.get_waic()
    bic=model.get_bic()
    summary = model.get_summary()
    summary['lag'] = i
    summary['waic'] = waic
    summary['bic']=bic
    df_summary = pd.concat([df_summary, summary])"""
#

In [72]:
model = Model(county_idxs, new_dat_mat, coords,categorical=True)    
trace,model_graph= model.fit_model(model_distribution='Poisson', draws=10000, step='Metropolis')
waic = model.get_waic()
summary = model.get_summary()
summary['waic'] = waic

In [73]:
summary[:30]

,mean,sd,hdi_3%,hdi_97%,waic
b_masking,-1.120,0.101,-1.300,-0.927,-5944.236
b_resid,0.004,0.010,-0.013,0.025,-5944.236
b_gp,-0.021,0.003,-0.026,-0.016,-5944.236
b_work,0.006,0.002,0.002,0.011,-5944.236
b_transit,0.003,0.001,0.000,0.005,-5944.236
b_rr,-0.000,0.003,-0.006,0.006,-5944.236
b_school[0],0.519,0.450,-0.169,1.238,-5944.236
b_school[1],0.042,0.043,-0.042,0.119,-5944.236
b_school[2],-0.041,0.030,-0.098,0.014,-5944.236
b_school[3],-0.142,0.025,-0.188,-0.096,-5944.236


In [ ]:
az.plot_trace(trace, var_names=['b_elderly', 'b_school', 'b_masking', 'b_pca_'])

In [ ]:
summary[['interpret_mean', 'interpret_hdi_3%','interpret_hdi_97%']] = summary[['mean', 'hdi_3%','hdi_97%']].apply(lambda x: (np.exp(x)-1)*100)
summary[:40]

In [ ]:
"""df_new=summary.loc[['b_pca_','b_masking']]
fig, ax = plt.subplots()

ax.errorbar(df_new.index, (np.exp(df_new['mean'])-1)*100, yerr=[(np.exp(df_new['mean'])-1)*100 - (np.exp(df_new['hdi_3%'])-1)*100, 
                                                                (np.exp(df_new['hdi_97%'])-1)*100 - (np.exp(df_new['mean'])-1)*100], fmt='o')
ax.set_xticklabels(df_new.index)
ax.set_xlabel('variables')
ax.set_ylabel('change in percent_positive %')
plt.show()"""

In [ ]:
"""#['b_pca_','b_masking','b_school[0]','b_school[1]','b_school[2]','b_school[3]',
                    #'b_elderly[0]','b_elderly[1]','b_elderly[2]','b_elderly[3]']
var_to_plot=['b_elderly[1]','b_elderly[2]','b_elderly[3]']
fig, ax = plt.subplots()

for variable, group in df_summary2.groupby('Variable'):
    if variable  in var_to_plot:
        ax.plot(group['lag'], (np.exp(group['mean'])-1)*100, label=variable)
for variable, group in df_summary2.groupby('Variable'):
    if variable in var_to_plot:
        ax.fill_between(group['lag'], (np.exp(group['hdi_3%'])-1)*100, (np.exp(group['hdi_97%'])-1)*100, alpha=0.1, color='blue')
        ax.plot(group['lag'], (np.exp(group['hdi_3%'])-1)*100, '--', color='black', alpha=0.5)
        ax.plot(group['lag'], (np.exp(group['hdi_97%'])-1)*100, '--', color='black', alpha=0.5)

ax.set_xlabel('Week (Lag)')
ax.set_ylabel('change_in_ili %')
ax.legend(title='Variable')
plt.show()"""

In [ ]:
ppc=model.get_posteriors()

In [ ]:
accuracies=[]
pred=ppc['y_like_county_idx']
print(pred.shape)
for i in range(1,51):
    state_idxs = np.where(new_dat_mat[:, 6] == i)[0]
    state_preds = pred[:, state_idxs]
    mean_pred=state_preds.mean(axis=0)
    observed=new_dat_mat[new_dat_mat[:, 6] == i,0]
    if len(observed)>0:
        r2 = r2_score(observed, mean_pred)
        accuracies.append(r2)
        print(f"Accuracy for index {i}: {r2}")
print(f"Average accuracy: {np.mean(accuracies)}")

In [ ]:
for idx in range(1,50):
    new_states=['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'GA', 'HI', 'ID',
           'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN',
           'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND',
           'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT',
           'VA', 'WA', 'WV', 'WI', 'WY']
    fig, ax = plt.subplots(figsize=(15,5))
    state_idxs = np.where(new_dat_mat[:, 6] == idx)[0]
    state_preds = pred[:, state_idxs]
    mean_pred=state_preds.mean(axis=0)
    observed=new_dat_mat[new_dat_mat[:, 6] == idx,0]
    plt.plot(df1[df1['state']==new_states[idx-1]]['ILIp']['2020-03-29':].index,mean_pred,label='prediction')
    plt.plot(df1[df1['state']==new_states[idx-1]]['ILIp']['2020-03-29':].index,observed,label='observed')
    plt.ylabel('ILIp')
    plt.legend()
    plt.title(f"{new_states[idx-1]}")
    plt.show()

In [ ]:

fig, ax = plt.subplots(figsize=(15,5))
plt.plot(df1[df1['state']==new_states[idx-1]]['ILIp']['2020-03-29':])

In [ ]:
"""def visualize_all_columns(df,ili):
    nrow=5
    ncol=2
    i=0
    fig, axes = plt.subplots(nrow, ncol,figsize=(15,15))
    fig.tight_layout()
    for r in range(nrow):
        for c in range(ncol):
            c1=ili
            c2=df[df.columns[i]]
            corr=round(c1.corr(c2),2)
            axes[r,c].plot(c1,label='il')
            axes[r,c].legend(['{:.2f}'.format(corr)],
                    loc = 'upper left',fontsize=20)
            ax2=axes[r,c].twinx()
            ax2.plot(c2,color='red',label=df.columns[i])
            ax2.legend()
            if i==len(df.columns)-1:
                break
            i+=1

def visualize_all_columns_outliers(df):
    nrow=5
    ncol=2
    i=0
    fig, axes = plt.subplots(nrow, ncol,figsize=(15,5))
    fig.tight_layout()
    for r in range(nrow):
        for c in range(ncol):
            axes[r,c].boxplot(df[df.columns[i]])
            axes[r,c].set_title(df.columns[i])
            if i==len(df.columns)-1:
                break
            i+=1
visualize_all_columns_outliers(df2_tx)"""